In [17]:
import pandas as pd

#Reading in raw tweets data
tweets = pd.read_csv("Tweets06192021.csv")
tweets.head()

,Unnamed: 0,Twitter Handle,Time,Content,Retweeting,Hashtags,Tagged,Links
0,0,RepJerryCarl,2021-06-18T18:38:29.000Z,"RT @BMAlliance: Thank you, @RepJerryCarl for c...",BMAlliance,$,$,$
1,1,RepJerryCarl,2021-06-18T15:45:03.000Z,"If you live in Mobile or Baldwin County, pleas...",$,al01,$,https://t.co/AcQ9JbBEHZ
2,2,RepJerryCarl,2021-06-17T23:39:56.000Z,@Truth_Gazette @CarlForAlabama Thank you!,$,$,"Truth_Gazette, CarlForAlabama",$
3,3,RepJerryCarl,2021-06-17T21:38:03.000Z,"@JonathanAHester Thank you, Jonathan!",$,$,JonathanAHester,$
4,4,RepJerryCarl,2021-06-17T21:37:45.000Z,@gopclassics Thank you!,$,$,gopclassics,$


In [21]:
import re
Retweeting = []
Tagged = []
Hashtags = []
Links = []


#Tries to grab where tweets were retweeted from and if they were not retweeted, finds hashtags, tags, and links in
#the tweet
for i in range(len(tweets['Twitter Handle'])):
    #I'm using the $ symbol to separate instances because Twitter usernames (and, to my knowledge, Hashtags) cannot
    #include special characters so the $ provides a good point to split along when creating documents for the graph
    #database while also making it easier to do the weird SQL regex matching to find whether a hashtag/user appears in a
    #given tweet by searching the column with a LIKE operator (ex: LIKE "%$Poutine$%" to find tweets with #Poutine)
    Retweet = ','
    Tags = ','
    Hashtag = ','
    Link = ','
    content = tweets['Content'][i]
    if content[:4] == 'RT @':
        Retweet += content[4:].split(':')[0] + "," #Has potential to break in really weird edge case but would almost require a congressperson to be actively screwing with me
    else:
        for j in range(len(content)):
            if content[j] == "@":
                Tags += re.sub('\W+', '', re.split(' |\n', content[j:])[0]) + ","
            elif content[j] == "#":
                Hashtag += re.sub('\W+', '', re.split(' |\n', content[j:])[0]) + ","
        words = re.split(' |\n', content)
        for word in words:
            if (len(word) > 4 and word[:4] == 'http'):
                Link += word + ','
    Retweeting.append(Retweet)
    Tagged.append(Tags)
    Hashtags.append(Hashtag)
    Links.append(Link)

In [22]:
tweets['Retweeting'] = Retweeting
tweets['Hashtags'] = Hashtags
tweets['Tagged'] = Tagged
tweets['Links'] = Links
tweets.head()

,Unnamed: 0,Twitter Handle,Time,Content,Retweeting,Hashtags,Tagged,Links
0,0,RepJerryCarl,2021-06-18T18:38:29.000Z,"RT @BMAlliance: Thank you, @RepJerryCarl for c...",",BMAlliance,",",",",",","
1,1,RepJerryCarl,2021-06-18T15:45:03.000Z,"If you live in Mobile or Baldwin County, pleas...",",",",al01,",",",",https://t.co/AcQ9JbBEHZ,"
2,2,RepJerryCarl,2021-06-17T23:39:56.000Z,@Truth_Gazette @CarlForAlabama Thank you!,",",",",",Truth_Gazette,CarlForAlabama,",","
3,3,RepJerryCarl,2021-06-17T21:38:03.000Z,"@JonathanAHester Thank you, Jonathan!",",",",",",JonathanAHester,",","
4,4,RepJerryCarl,2021-06-17T21:37:45.000Z,@gopclassics Thank you!,",",",",",gopclassics,",","


In [23]:
#Drop weird index column that appears when loading from a csv that you saved
tweets = tweets.iloc[:,1:]
tweets.head()

,Twitter Handle,Time,Content,Retweeting,Hashtags,Tagged,Links
0,RepJerryCarl,2021-06-18T18:38:29.000Z,"RT @BMAlliance: Thank you, @RepJerryCarl for c...",",BMAlliance,",",",",",","
1,RepJerryCarl,2021-06-18T15:45:03.000Z,"If you live in Mobile or Baldwin County, pleas...",",",",al01,",",",",https://t.co/AcQ9JbBEHZ,"
2,RepJerryCarl,2021-06-17T23:39:56.000Z,@Truth_Gazette @CarlForAlabama Thank you!,",",",",",Truth_Gazette,CarlForAlabama,",","
3,RepJerryCarl,2021-06-17T21:38:03.000Z,"@JonathanAHester Thank you, Jonathan!",",",",",",JonathanAHester,",","
4,RepJerryCarl,2021-06-17T21:37:45.000Z,@gopclassics Thank you!,",",",",",gopclassics,",","


In [25]:
#Save new features to the csv
tweets.to_csv("Tweets06192021.csv")